In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
%store -r model
%store -r X
%store -r y
%store -r X_test
%store -r y_test
%store -r CNN_LSTM_model

In [3]:
from tensorflow.keras.models import Model

pretrained_model = CNN_LSTM_model
# Xóa lớp cuối cùng của mô hình đã được huấn luyện trước đó
pretrained_model.pop()

# Thêm lớp fully connected (Dense Layer) cuối cùng với số lớp phân loại mới
num_new_classes = 5  # Số lớp ứng dụng mới cần phân loại
pretrained_model.add(keras.layers.Dense(num_new_classes, activation='softmax', name='new_dense_layer'))

# Đóng băng trọng số của các lớp trong mô hình đã được huấn luyện trước đó
for layer in pretrained_model.layers[:-1]:
    layer.trainable = False

# In thông tin mô hình mới
pretrained_model.summary()

# Biên dịch mô hình mới
pretrained_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 20, 32)            20512     
                                                                 
 max_pooling1d (MaxPooling1  (None, 10, 32)            0         
 D)                                                              
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 new_dense_layer (Dense)     (None, 5)                 165       
                                                        

In [4]:
pretrained_model.fit(X, y, epochs=10, batch_size=16)

Epoch 1/10
823/823 [==============================] - 5s 4ms/step - loss: 0.9481 - accuracy: 0.7633
Epoch 2/10
823/823 [==============================] - 3s 4ms/step - loss: 0.3888 - accuracy: 0.8680
Epoch 3/10
823/823 [==============================] - 3s 4ms/step - loss: 0.3565 - accuracy: 0.8750
Epoch 4/10
823/823 [==============================] - 3s 4ms/step - loss: 0.3377 - accuracy: 0.8795
Epoch 5/10
823/823 [==============================] - 3s 4ms/step - loss: 0.3266 - accuracy: 0.8818
Epoch 6/10
823/823 [==============================] - 3s 4ms/step - loss: 0.3246 - accuracy: 0.8833
Epoch 7/10
823/823 [==============================] - 3s 4ms/step - loss: 0.3119 - accuracy: 0.8860
Epoch 8/10
823/823 [==============================] - 3s 4ms/step - loss: 0.3047 - accuracy: 0.8862
Epoch 9/10
823/823 [==============================] - 3s 4ms/step - loss: 0.3031 - accuracy: 0.8873
Epoch 10/10
823/823 [==============================] - 3s 4ms/step - loss: 0.3005 - accuracy: 0.8880

In [5]:
import numpy as np

# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = pretrained_model.evaluate(X_test, y_test)
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = pretrained_model.predict(X_test)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)

412/412 [==============================] - 2s 5ms/step - loss: 0.2954 - accuracy: 0.8922
Độ chính xác trên tập kiểm tra: 0.8922224044799805
412/412 [==============================] - 2s 4ms/step
Ma trận nhầm lẫn:
[[2842    0    0    0    0]
 [   0 1714    2   50  268]
 [   0    0 2475    2  331]
 [   0   94    2 1574   70]
 [   0  166  421   13 3142]]
Báo cáo phân loại:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2842
           1       0.87      0.84      0.86      2034
           2       0.85      0.88      0.87      2808
           3       0.96      0.90      0.93      1740
           4       0.82      0.84      0.83      3742

    accuracy                           0.89     13166
   macro avg       0.90      0.89      0.90     13166
weighted avg       0.89      0.89      0.89     13166

